## Using Visdom With FastAI

In this notebook i will show how to use Visdom https://github.com/facebookresearch/visdom with the FastAI library

Visdom is an amazing data visualization tool. What it essentially does is to create a new server (basically opens a new port in the server) and displays all the data in that place. There are a wide range of options and a lot of data type support, lot of plotting library support. The UI is extremely clean. I highly encourage to have a look at the github repository to have a taste of its capabilities.

In this notebook, we will try to plot the training loss after every batch and see how that varies. Next, we will plot the training loss and the validation loss after every epoch. This notebook should give a broad and general idea and one can incorporate other types of visualization into this easily.

## Confirming Visdom is working properly

As always, we will first ensure that visdom is working properly. Easiest way to install visdom is via `pip install visdom`.

Next, in your terminal open a tmux session or screen session and open a visdom server with suitable port (by default it uses 8097). In this example I use the port 6008. 

In terminal session type: `python -m visdom.server -port=6008`

In [2]:
import visdom
import numpy as np
vis = visdom.Visdom(port=6008)
vis.text('Hello, world!')
vis.image(np.ones((3, 10, 10)))

'window_365889b2226a26'

Now head over to `localhost:6008` (or ipaddress-of-server:6008) and select `main` in the environments. You should be able to see "Hello World" in a canvas. Once done, do a kernel restart so that the port 6008 is free again

It might be a good idea to do port forwarding (tuneling) of another port other than the one you use jupyter notebook for this purpose. In my case, I have forward 6007 and 6008 ports for jupyter and visdom respectively.

Now we will try to use Visdom to monitor training loss. Again, I will be using dogcats dataset which can be found here http://files.fast.ai/data/.

In [1]:
import matplotlib
matplotlib.use('Agg')

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [7]:
import visdom

In [4]:
PATH = "../data/dogscats/"
sz=224

In [5]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz))

To use Visdom properly we will first have look at the concept of CallBacks. I don't think I can explain callbacks better than that explained in this post https://github.com/sgugger/Deep-Learning/blob/master/Using%20the%20callback%20system%20in%20fastai.ipynb by Sylvain Gugger.

It is a good idea to create plotter class for plotting onto visdom. The following function is inspired from https://github.com/andreasveit/triplet-network-pytorch/blob/master/train.py

In [14]:
class VisdomLinePlotter:
    def __init__(self, env='main'):
        self.vis = visdom.Visdom(port=6008)
        self.env = env
        self.plots = dict()
        
    def plot(self, window_name, var_name, x, y, xlabel='Epochs', ylabel='Loss'):
        if window_name not in self.plots:
            self.plots[window_name] = self.vis.line(X=np.array([x]), Y=np.array([y]), 
                                                    env=self.env, opts=dict(
                                                        legend=[var_name],
                                                        title=window_name,
                                                        xlabel=xlabel,
                                                        ylabel=ylabel
                                                    ))
        else:
            self.vis.line(X=np.array([x]), Y=np.array([y]), env=self.env, 
                          win=self.plots[window_name], name=var_name, update='append')

We will define a callback function which inherits the Callback class.

In [18]:
class visdom_callback(Callback):
    def __init__(self, plotter):
        self.plotter = plotter
        self.num_epochs=0
        self.num_batches=0
    def on_batch_end(self, los):
        self.deb_loss = los
        self.num_batches+=1
        self.plotter.plot('train', 'train_loss', self.num_batches, 
                          self.deb_loss, xlabel='Batches', ylabel='Training Loss')

In [15]:
plotter = VisdomLinePlotter()

vcb = visdom_callback(plotter)

In [11]:
learn = ConvLearner.pretrained(arch, data)

While the next line executes, you should be able to see at `localhost:6008` or `ipaddress:6008` the training plot being continuously updated

In [17]:
learn.fit(1e-2, 1, cycle_len=1, metrics=[accuracy], callbacks=[vcb])

epoch      trn_loss   val_loss   accuracy                     
    0      0.0607     0.028024   0.9895    



[array([0.02802]), 0.9895]

Now we will try to see the training loss and validation loss after each epoch. We will need to modify our callback a bit

In [19]:
class visdom_callback(Callback):
    def __init__(self, plotter):
        self.plotter = plotter
        self.num_epochs=0
        self.num_batches=0
    def on_batch_end(self, los):
        self.deb_loss = los
        self.num_batches+=1
    def on_epoch_end(self, metrics):
        self.num_epochs += 1
        self.plotter.plot('train', 'train_loss', self.num_epochs, 
                          self.deb_loss, xlabel='Epochs', ylabel='Loss')
        self.plotter.plot('train', 'val_loss', self.num_epochs, 
                          metrics[0], xlabel='Epochs', ylabel='Loss')

In [20]:
plotter = VisdomLinePlotter()

vcb = visdom_callback(plotter)

To speed up the process we can try only fine tuning the final layers. For this we can save the activations first, then freeze the layers and set precompute as true

In [21]:
learn.save_fc1()

100%|██████████| 32/32 [00:03<00:00,  8.61it/s]


In [23]:
learn.freeze()
learn.precompute = True

Again you can see your visdom plots while this is training on the visdom port.

In [24]:
learn.fit(1e-2, 10, cycle_len=1, metrics=[accuracy], callbacks=[vcb])

epoch      trn_loss   val_loss   accuracy                     
    0      0.045204   0.024696   0.9915    
    1      0.037385   0.024245   0.992                        
    2      0.037161   0.024644   0.9915                       
    3      0.041269   0.025129   0.991                        
    4      0.040194   0.02618    0.991                        
    5      0.043403   0.026201   0.9915                       
    6      0.037505   0.028432   0.9895                       
    7      0.033672   0.027222   0.9905                       
    8      0.026833   0.027428   0.9905                       
    9      0.030505   0.026051   0.99                         



[array([0.02605]), 0.99]

Cheers. Now you can easily customize visdom and use it in your own project which uses fastai library.